In [1]:
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import pandas as pd
import numpy as np

In [9]:
# read in the phono from the pickle so that we have the associated CMU pronunciations
all_tokens_phono = pd.read_pickle('prov_csv/pvd_utt_glosses_phono_cleaned_inflated_to_next_notebook.pkl')

In [10]:
excludes = ['', '*','(.)','(..)', '(...)','(....)','(.....)']
phono = all_tokens_phono.loc[~all_tokens_phono.actual_phonology.isin(excludes) & 
                             ~all_tokens_phono.model_phonology.isin(excludes)]
phono.shape

(222881, 32)

In [11]:
phono.iloc[0]

token                             mommy
utterance_id                   16759315
gloss                             Mommy
transcript_id                     42204
utterance_order                       6
target_child_name                  Alex
speaker_code                        CHI
type                        declarative
punct                                 .
speaker_code_simple               [CHI]
gloss_with_punct           [CHI] mommy.
token_id                          20565
seq_utt_id                            3
actual_phonology                    ɑmɪ
model_phonology                   mɑmiː
target_child_age                  514.0
bert_token_id                        23
model_phonology_clean              mɑmi
actual_phonology_clean              ɑmə
model_phonology_no_dia             mɑmi
actual_phonology_no_dia             ɑmə
cv_raw_actual                       vcv
cv_collapsed_actual                 vcv
num_vowels_actual                   2.0
cv_raw_model                       cvcv


In [12]:
cmu = pd.read_pickle('phon/cmu_in_childes.pkl')
cmu.head(5)

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
70,71,a,AH0,[AH],[ə],[v],1,ə
71,72,a,EY1,[EY],[eɪ],[v],1,eə
77,78,aa,EY2 EY1,"[EY, EY]","[eɪ, eɪ]","[v, v]",2,eəeə
92,93,aaron,EH1 R AH0 N,"[EH, R, AH, N]","[ɛ, ɹ, ə, n]","[v, c, v, c]",2,ɛɹən
102,103,ab,AE1 B,"[AE, B]","[æ, b]","[v, c]",1,æb


In [13]:
phono['model_ipa'] = phono.model_phonology_no_dia
phono['actual_ipa'] = [' '.join(list(x)) for x in phono.actual_phonology_no_dia]

/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
'a' in set(list(''.join(phono['actual_ipa'])))

True

In [17]:
cmu_map = dict(zip(cmu.word, cmu.ipa_short))
def cmu_mapper(x, cmu_map):
    token = x[0]
    model = x[1]
    if x[0] in cmu_map:
        return(cmu_map[x[0]])
    else:
        return(x[1])


phono['cmu'] = [cmu_mapper(x, cmu_map) for x in zip(phono.token, phono.model_ipa)]

/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [18]:
phono[['cmu','actual_ipa']]

,cmu,actual_ipa
23,mɑmi,ɑ m ə
125,wi,w i
151,wi,w i
189,wu,u
685,əɹni,ɛ
...,...,...
2991828,noʊbɑdi,n o b ɑ ɾ i
2991829,heəts,h e ə s
2991834,oʊ,o
2991835,hwaə,w a ə


In [20]:
num_dev = 20000
num_train = 180000
num_test = 20000

phono[['cmu','actual_ipa']].iloc[0:num_dev].to_csv('fst_training/dev/chi_dev.tsv',
    sep='\t', header=False, index=False)
phono[['cmu','actual_ipa']].iloc[num_dev:num_dev+num_train].to_csv('fst_training/train/chi_train.tsv',
    sep='\t', header=False, index=False)
phono[['cmu','actual_ipa']].iloc[num_dev+num_train:num_dev+num_train+num_test].to_csv('fst_training/test/chi_test.tsv',
    sep='\t', header=False, index=False)

In [21]:
phono[['model_ipa_split']] = [' '.join(list(x)) for x in phono['cmu']]
phono[['model_ipa_split']].iloc[num_dev:num_dev+num_train].to_csv(
    'fst_training/train/chi_inputs.tsv',
    sep='\t', header=False, index=False)

/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/stephan/notebooks/child-directed-listening/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
# then run ./sweep from inside the sigmorphon phoneme-to-grapheme codebase